# 라즈베리파이 실시간 전압/전류 측정

## ⚙ 구성
|항목|모델명|용도|
|-------|-------|-------|
전압 측정|ZMPT101B|220V 전압 감지(출력: 아날로그)
전류 측정|ACS712 (5A/20A/30A)|부하 전류 감지 (출력: 아날로그)
ADC|ADS1115|아날로그 센서 → 디지털 신호로 변환
저장 형식|CSV|전처리 용이, 속도 빠름
측정 주기|100Hz(10ms 간격, 일반 논문에서 자주 사용)

## 🧠 흐름
- ZMPT101B & ACS712 (Analog)
- ADS1115 (I2C → Raspberry Pi)
- 실시간 측정 (0.01초 간격)
- CSV 파일로 저장

## ⚠️ 참고사항
- ACS712 전류 변환 공식: `current (A) = (measured_voltage - Vcc/2) / sensitivity` (예: 20A 모델은 100mV/A, Vcc=5V일 때 2.5V 중심 기준)
- ZMPT101B도 보정이 필요하지만, 일단 상대적 변화 감지용으로도 사용 가능

In [ ]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd

# I2C 초기화
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1

# 센서 채널 설정
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

# 보정 파라미터 
ZMPT_offset = 1.65  # 중심값 (ZMPT101B 기준)
ZMPT_scale = 57.14  # 1.65V → 약 220V일 때, 실험해서 보정 필요

ACS712_offset = 2.5      # 0A일 때 기준 전압
ACS712_sensitivity = 0.1 # 20A 버전: 100mV/A



data = []

try:
    while True:
        now = time.time()

        # 원시 센서값
        v_raw = voltage_ch.voltage
        c_raw = current_ch.voltage

        # 보정된 실제값 계산
        real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        real_current = (c_raw - ACS712_offset) / ACS712_sensitivity

        # 로그 저장
        data.append([now, v_raw, c_raw, real_voltage, real_current])
        # CSV 저장 (100줄씩)
        if len(data) >= 100:
            df = pd.DataFrame(data, columns=['timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)'])
            # df.to_csv("calibrated_data.csv", mode='a', index=False, header=False)
            print(df)
            data = []

        time.sleep(0.0001)

except KeyboardInterrupt:
    print("측정 중단됨")
